In [2]:
import numpy as np
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import time
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cpu


In [4]:
data_og = np.load('Data/data_train.npy').T
labels_og = np.loadtxt('Data/correct_labels.npy',delimiter=',')

train_data, test_data, train_labels, test_labels = train_test_split(data_og, labels_og, test_size=0.2, random_state=42)

nimages = len(train_labels)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/data_train.npy'

In [ ]:
class data_and_labels(Dataset):
    def __init__(self,data,labels,transform = None):
        self.transform = transform
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,index):
        image = self.data[index,:]
        image = image.reshape(300,300)
        label = torch.tensor(int(self.labels[index]))
        if self.transform:
            image = self.transform(image)
        return (image,label)

In [ ]:
dataset = data_and_labels(train_data,train_labels,transform = transforms.ToTensor())
print(np.shape(train_data))
print(nimages)
print(int(nimages*0.8))
print(nimages-int(nimages*0.8))
train_set,valid_set = torch.utils.data.random_split(dataset,[int(nimages*0.8),nimages-int(nimages*0.8)])

(5376, 90000)
5376
4300
1076


In [ ]:
train_loader = DataLoader(dataset=train_set,batch_size=20,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset=valid_set,batch_size=20,shuffle=True,num_workers=2)

lr = 0.001
momentum = 0.9
num_epochs = 20

In [ ]:
#conv1
c1inputsize = 1 #greyscale
c1outputsize = 3 #idk
c1kernelsize = 5 #idk
#pool1
p1ks = 5 #kernel size
p1s = p1ks #stride size
#conv2
c2is = c1outputsize
c2os = 10
c2ks = 5
#fully connected 1
fc1is = c2os*c1kernelsize*c2ks
fc1os = 80
#fully connected 2
fc2is = fc1os
fc2os = 40
#fully connected 3
fc3is = fc2os
fc3os = 22

model = nn.Sequential(
    nn.Conv2d(c1inputsize,c1outputsize,c1kernelsize),
    nn.MaxPool2d(p1ks,p1s),
    nn.ReLU(),
    nn.Conv2d(c2is,c2os,c2ks),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(fc1is,fc1os),
    nn.Linear(fc2is,fc2os),
    nn.Linear(fc3is,fc3os)
)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)

In [ ]:
train_losses = []
valid_losses = []

for epoch in range(1,num_epochs+1):
    train_loss=0.0
    valid_loss=0.0
    model.train()
    for image, label in train_loader:
        image = image.cuda()
        label = label.cuda()
        optimizer.zero_grad()
        prediction=  model(image)
        loss = loss_fn(prediction,label)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()*image.size(0)
    model.eval()
    for image,label in test_loader:
        image = image.cuda()
        label = label.cuda()
        prediction = model(image)
        loss = loss_fn(prediction,label)
        valid_loss+=loss.item()*image.size(0)
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(test_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print('Epoch:{} Train Loss:{:.4f} valid Loss:{:.4f}'.format(epoch,train_loss,valid_loss))